In [1]:
import fastai
import pickle
from fastai.utils.mem import *

In [2]:
stage = 'stage_2'
data_dir = Path('data')
fn_to_study_ix = pickle.load(open(f'{data_dir}/{stage}_fn_to_study_ix.pickle', 'rb'))
study_ix_to_fn = pickle.load(open(f'{data_dir}/{stage}_study_ix_to_fn.pickle', 'rb'))
fn_to_labels = pickle.load(  open(f'{data_dir}/{stage}_train_fn_to_labels.pickle', 'rb'))

In [3]:
classes = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
labels = []
for fn, lbls in fn_to_labels.items():
    row = {'fn':fn,'study':fn_to_study_ix[fn][0]}
    for c in classes:
        row[c] = c in fn_to_labels[fn]
    labels.append(row)

labels_df = pd.DataFrame(labels)
labels_df.head() 

,fn,study,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_76d55d9d0,ID_0000298a7d,False,False,False,False,False,False
1,ID_96d282ea9,ID_0000298a7d,False,False,False,False,False,False
2,ID_7d8a7c29d,ID_0000298a7d,False,False,False,False,False,False
3,ID_4d4401491,ID_0000298a7d,False,False,False,False,False,False
4,ID_8f5ded0b7,ID_0000298a7d,False,False,False,False,False,False


In [4]:
study_labels = labels_df.groupby('study').agg('sum').reindex()
for c in classes :
    study_labels[c+'_size'] = 0 # 0 :not present, 1:small, 2:big
    c_idx = study_labels.query(f'{c}>0').sort_values(c).index
    c_small, c_big = c_idx[:len(c_idx)//2], c_idx[len(c_idx)//2:]
    study_labels.loc[c_small,c+'_size'] = 1
    study_labels.loc[c_big,c+'_size'] = 2
size_classes = [c+'_size' for c in classes]
study_labels['strat_class'] = study_labels[size_classes].astype(str).sum(axis=1)
study_labels.sample(10)

,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any_size,epidural_size,intraparenchymal_size,intraventricular_size,subarachnoid_size,subdural_size,strat_class
study,,,,,,,,,,,,,
ID_0c828b6688,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_fe788cb8e6,6.0,0.0,2.0,0.0,4.0,0.0,1,0,1,0,1,0,101010.0
ID_676bad1cd8,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_d0df6455dc,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_db74a340fd,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_29ecf20f76,7.0,0.0,5.0,5.0,0.0,0.0,1,0,1,1,0,0,101100.0
ID_2d80aed85a,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_d9cf56413e,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0
ID_a2d95718f5,15.0,0.0,0.0,0.0,0.0,15.0,2,0,0,0,0,2,200002.0


In [5]:
from sklearn.model_selection import StratifiedKFold

#this will throw a warning that can be ignored
skf = StratifiedKFold(n_splits=19, shuffle=True, random_state=1972)
study_labels['fold'] = -1
for fold, (oof_idx,f_idx) in enumerate(skf.split(study_labels, study_labels.strat_class)):
    study_labels.loc[study_labels.iloc[f_idx].index, 'fold' ] = fold
study_labels.head()

/home/antor/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=19.
  % (min_groups, self.n_splits)), UserWarning)


,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any_size,epidural_size,intraparenchymal_size,intraventricular_size,subarachnoid_size,subdural_size,strat_class,fold
study,,,,,,,,,,,,,,
ID_0000298a7d,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,6
ID_0004c4e54c,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,15
ID_000a935543,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,2
ID_000f6fd7db,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,9
ID_0010b2528e,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,6


In [6]:
study_to_data = {}
for study in study_ix_to_fn.keys():
    if study in study_labels.index:
        study_to_data[study] = {'fold': study_labels.loc[study].fold}
    else:
        study_to_data[study] = {'fold': -1} #study not in label set
pickle.dump(study_to_data, open(f"data/{stage}_study_to_data.pickle", "wb" ))